In [ ]:
data_root = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/tld_db/train/images'  # 이미지 경로
csv_path = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/test_csv/Co-DETR(SwinL, lsj, 3ep)_val.csv'  # CSV 파일 경로
json_path = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/tld_db/json/val_coco.json'  # 정답지 경로

In [2]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_curve, average_precision_score

# 설정
json_path = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/tld_db/json/val_coco.json'  # 정답지 경로
predictions_path = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/test_csv/Co-DETR(SwinL, lsj, 3ep)_val.csv'  # 예측 결과 경로

class_names = {
    0: "veh_go", 1: "veh_goLeft", 2: "veh_noSign", 3: "veh_stop", 4: "veh_stopLeft", 
    5: "vef_stopWarning", 6: "veh_warning", 7: "ped_go", 8: "ped_noSign", 9: "ped_stop", 
    10: "bus_go", 11: "bus_noSign", 12: "bus_stop", 13: "bus_warning"
}

class_colors = {
    0: (1.0, 0.0, 0.0),    # 빨간색
    1: (0.0, 1.0, 0.0),    # 초록색
    2: (0.0, 0.0, 1.0),    # 파란색
    3: (1.0, 1.0, 0.0),    # 노란색
    4: (1.0, 0.0, 1.0),    # 보라색
    5: (0.0, 1.0, 1.0),    # 청록색
    6: (0.5, 0.0, 0.0),    # 어두운 빨간색
    7: (0.0, 0.5, 0.0),    # 어두운 초록색
    8: (0.0, 0.0, 0.5),    # 어두운 파란색
    9: (0.5, 0.5, 0.0),    # 어두운 노란색
    10: (0.5, 0.0, 0.5),   # 어두운 보라색
    11: (0.0, 0.5, 0.5),   # 어두운 청록색
    12: (0.25, 0.25, 0.25),# 회색
    13: (0.5, 0.5, 0.5)    # 밝은 회색
}

# JSON 정답지 로드
def load_ground_truth(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    ground_truth = {}
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        class_id = annotation['category_id']
        bbox = annotation['bbox']  # 형식: [x, y, width, height]
        
        if image_id not in ground_truth:
            ground_truth[image_id] = []
        ground_truth[image_id].append((class_id, bbox))
    return ground_truth

# CSV 예측 데이터 로드 및 파싱
def load_predictions(predictions_path):
    predictions = {}
    df = pd.read_csv(predictions_path, header=None)
    for _, row in df.iterrows():
        image_name = int(row[1].split(".")[0])  # 파일명에서 이미지 ID 추출
        items = row[0].split()
        
        image_predictions = []
        for i in range(0, len(items), 6):
            class_id = int(items[i])
            score = float(items[i + 1])
            x_min = float(items[i + 2])
            y_min = float(items[i + 3])
            x_max = float(items[i + 4])
            y_max = float(items[i + 5])
            image_predictions.append((class_id, score, x_min, y_min, x_max, y_max))
        
        predictions[image_name] = image_predictions
    return predictions

# mAP50 및 PR Curve 계산
def calculate_map50_and_pr(ground_truth, predictions):
    all_aps = []
    precisions, recalls = {}, {}
    
    for class_id in class_names.keys():
        y_true, y_scores = [], []
        
        for img_id, gt_boxes in ground_truth.items():
            gt_classes = [gt[0] for gt in gt_boxes if gt[0] == class_id]
            pred_scores = [pred[1] for pred in predictions.get(img_id, []) if pred[0] == class_id]
            
            y_true.extend([1] * len(gt_classes) + [0] * len(pred_scores))
            y_scores.extend(pred_scores + [0] * len(gt_classes))
        
        if len(set(y_true)) > 1:
            precision, recall, _ = precision_recall_curve(y_true, y_scores)
            ap = average_precision_score(y_true, y_scores)
            all_aps.append(ap)
            
            precisions[class_id] = precision
            recalls[class_id] = recall
        else:
            precisions[class_id] = []
            recalls[class_id] = []
            all_aps.append(0.0)

    mean_ap = np.mean(all_aps)
    return precisions, recalls, all_aps, mean_ap

# 시각화 함수
def plot_pr_curve(precisions, recalls, all_aps, mean_ap):
    plt.figure(figsize=(14, 10))
    
    for class_id, color in class_colors.items():
        if len(precisions[class_id]) > 0 and len(recalls[class_id]) > 0:
            plt.plot(recalls[class_id], precisions[class_id], label=f"{class_names[class_id]}: AP50={all_aps[class_id]:.3f}", color=color)
    
    plt.title(f"PR Curve by Class | Mean mAP50 = {mean_ap:.3f}", fontsize=16, weight='bold')
    plt.xlabel("Recall", fontsize=14)
    plt.ylabel("Precision", fontsize=14)
    plt.legend(loc='best', fontsize=10)
    plt.grid(True)
    plt.show()

# 실행
ground_truth = load_ground_truth(json_path)
predictions = load_predictions(predictions_path)

precisions, recalls, all_aps, mean_ap = calculate_map50_and_pr(ground_truth, predictions)
plot_pr_curve(precisions, recalls, all_aps, mean_ap)

ValueError: invalid literal for int() with base 10: 'image_id'

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patheffects import withStroke
import pandas as pd
import json

# 설정
data_root = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/tld_db/train/images'  # 이미지 경로
csv_path = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/test_csv/Co-DETR(SwinL, lsj, 3ep)_val.csv'  # CSV 파일 경로
json_path = '/Users/imch/workspace/2024-Autonomous-Driving-Artificial-Intelligence-Challenge/tld_db/json/val_coco.json'  # 정답지 경로

class_names = { 
    0: "veh_go", 1: "veh_goLeft", 2: "veh_noSign", 3: "veh_stop", 4: "veh_stopLeft", 
    5: "vef_stopWarning", 6: "veh_warning", 7: "ped_go", 8: "ped_noSign", 9: "ped_stop", 
    10: "bus_go", 11: "bus_noSign", 12: "bus_stop", 13: "bus_warning"
}  # 클래스 이름

# 클래스별 색상 설정
class_colors = {
    0: (255, 0, 0),    # 빨간색
    1: (0, 255, 0),    # 초록색
    2: (0, 0, 255),    # 파란색
    3: (255, 255, 0),  # 노란색
    4: (255, 0, 255),  # 보라색
    5: (0, 255, 255),  # 청록색
    6: (128, 0, 0),    # 어두운 빨간색
    7: (0, 128, 0),    # 어두운 초록색
    8: (0, 0, 128),    # 어두운 파란색
    9: (128, 128, 0),  # 어두운 노란색
    10: (128, 0, 128), # 어두운 보라색
    11: (0, 128, 128), # 어두운 청록색
    12: (64, 64, 64),  # 회색
    13: (128, 128, 128) # 밝은 회색
}

# JSON 정답지 로드 함수
def load_ground_truth(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    ground_truth = {}
    for annotation in data['annotations']:
        image_id = annotation['image_id']
        bbox = annotation['bbox']  # [x, y, width, height] 형식
        class_id = annotation['category_id']
        
        # 이미지 ID별로 정답 데이터를 리스트 형태로 저장
        if image_id not in ground_truth:
            ground_truth[image_id] = []
        ground_truth[image_id].append((class_id, bbox))
    
    return ground_truth

# CSV 파일 로드 및 파싱 함수
def parse_csv_data(csv_path):
    df = pd.read_csv(csv_path, header=None)
    data = []

    for _, row in df.iterrows():
        if row[0] == 'PredictionString':  # 헤더 무시
            continue
        image_name = row[1]
        predictions = []
        items = row[0].split()
        for i in range(0, len(items), 6):
            class_id = int(items[i])
            score = float(items[i + 1])
            x_min = float(items[i + 2])
            y_min = float(items[i + 3])
            x_max = float(items[i + 4])
            y_max = float(items[i + 5])
            predictions.append((class_id, score, x_min, y_min, x_max, y_max))
        data.append((image_name, predictions))
    return data

# 텍스트 위치 자동 조정 함수
def find_non_overlapping_position(x1, y1, x2, y2, ax, default_position):
    position_options = [
        (x1, y1 - 10),  # 상단 좌측
        (x2, y1 - 10),  # 상단 우측
        (x1, y2 + 20),  # 하단 좌측
        (x2, y2 + 20)   # 하단 우측
    ]
    for pos in position_options:
        if 0 <= pos[0] < ax.get_xlim()[1] and 0 <= pos[1] < ax.get_ylim()[0]:
            return pos
    return default_position

# 예측 데이터 시각화 함수
def visualize_predictions(image_path, predictions, image_name):
    fig, axes = plt.subplots(1, 2, figsize=(18, 9), facecolor='#2f2f2f')
    h, w, _ = cv2.imread(image_path).shape

    for i, (ax, threshold) in enumerate(zip(axes, [0.5, 0.5])):
        # 이미지 읽기 및 설정
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.axis('off')
        ax.set_title(f"{image_name} | {'Score >= 0.5' if i == 0 else 'Score < 0.5'}",
                     fontsize=16, color='white', weight='bold')

        # score에 따른 분리된 예측값들
        filtered_predictions = [pred for pred in predictions if (pred[1] >= threshold) == (i == 0)]
        top_predictions = sorted(filtered_predictions, key=lambda x: x[1], reverse=True)[:5]

        for idx, (class_id, score, x_min, y_min, x_max, y_max) in enumerate(filtered_predictions):
            color = class_colors.get(class_id, (255, 255, 255))
            color_with_opacity = (color[0] / 255, color[1] / 255, color[2] / 255, 0.6)
            rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor=color_with_opacity, facecolor='none')
            ax.add_patch(rect)

            if (class_id, score, x_min, y_min, x_max, y_max) in top_predictions:
                default_position = (x_min, y_min - 10)
                text_position = find_non_overlapping_position(x_min, y_min, x_max, y_max, ax, default_position)
                ax.text(
                    text_position[0], text_position[1], f"{class_id}: {score:.2f}",
                    color='white', fontsize=8, weight='bold',
                    bbox=dict(facecolor=color_with_opacity, alpha=0.4, edgecolor='none', pad=0.1),
                    path_effects=[withStroke(linewidth=1, foreground="black")]
                )

    patches = [mpatches.Patch(color=[c/255 for c in color], label=f"{class_id}: {class_names[class_id]}")
               for class_id, color in class_colors.items()]
    plt.legend(handles=patches, loc='center left', bbox_to_anchor=(1, 0.5), frameon=False, fontsize=12, labelcolor='white')
    plt.tight_layout(pad=2)
    plt.show()

# 실행
ground_truth = load_ground_truth(json_path)  # JSON 정답지 로드
print(f"총 {len(ground_truth)}개의 이미지에 대한 정답 데이터가 로드되었습니다.")

data = parse_csv_data(csv_path)
for image_name, predictions in data:
    image_path = os.path.join(data_root, image_name)
    if os.path.exists(image_path):
        visualize_predictions(image_path, predictions, image_name)